In [ ]:
import rosbag
import csv
import os
import sys

In [ ]:
####################################################
########## Modify the Configuration  Here ##########
####################################################

# Replace with your .bag file path
bag_file = r"./imus.bag"  

# Replace with the topics you want to extract
topics = [
    '/xsens/imu',  
    '/obd2/speed'
]

# Replace with your output folder path.
# The script will create csv file for every topic
out_folder = r"./"

In [ ]:
def extract_msg_fields(msg, prefix=''):
    fields = {}
    for field_name in msg.__slots__:
        field_value = getattr(msg, field_name)
        
        # Check if the field is a ROS message by checking for __slots__ attribute
        if hasattr(field_value, '__slots__'):
            # Recurse into nested ROS messages
            fields.update(extract_msg_fields(field_value, prefix + field_name + '.'))
        else:
            # Otherwise, add the field directly (primitive type or list of primitives)
            fields[prefix + field_name] = field_value

    return fields


In [ ]:
def extract_data(bag_file, topics,out_folder):
    if not os.path.isfile(bag_file):
        print(f"Error: Bag file '{bag_file}' does not exist.")
        return

    
    print("opening :",bag_file)
    bag = rosbag.Bag(bag_file, 'r')

    # Check if the topics exist in the bag file
    available_topics = bag.get_type_and_topic_info()[1].keys()
    missing_topics = [topic for topic in topics if topic not in available_topics]
    if missing_topics:
        print(f"Error: The following topics are not available in the bag file: {', '.join(missing_topics)}")
        bag.close()
        return

    #bag timing info
    start_time = bag.get_start_time()
    end_time = bag.get_end_time()
    total_duration = end_time - start_time

    # Open CSV writers for each topic
    csv_writers = {}
    for topic in topics:
        csv_file = topic.replace('/', '_') + '.csv'
        csv_file = csv_file.lstrip('_')
        csv_file_path = os.path.join(out_folder,csv_file)
        
        csv_writers[topic] = {
            "path":csv_file_path,
            "file": open(csv_file_path, 'w', newline=''),
            "writer": None  
        }

    last_time = start_time
    for topic, msg, t in bag.read_messages(topics=topics):
        # Calculate progress
        current_time = t.to_sec()
        elapsed_time = current_time - start_time
        progress = (elapsed_time / total_duration) * 100

        # Print progress Every second
        if current_time - last_time >= 1: 
            sys.stdout.write(f"\rProcessing bag: {progress:.2f}% complete")
            sys.stdout.flush()
            last_time = current_time


        # Extract the message fields
        msg_dict = {'time': t.to_sec()}
        msg_dict.update(extract_msg_fields(msg))

        # Initialize the CSV writer if not already done
        if csv_writers[topic]["writer"] is None:
            csv_writers[topic]["writer"] = csv.DictWriter(csv_writers[topic]["file"], fieldnames=msg_dict.keys())
            csv_writers[topic]["writer"].writeheader()

        # Write the message data
        csv_writers[topic]["writer"].writerow(msg_dict)

       
    #Close the bag
    bag.close()
    sys.stdout.write(f"\rProcessing bag: 100.0% complete")
    sys.stdout.flush()
    print()


    # Close all CSV files
    print("---------------------------------------------")
    for topic, writer_info in csv_writers.items():
        writer_info["file"].close()
        print(f"Data from topic '{topic}' saved to '{writer_info['path']}'")



In [ ]:
extract_data(bag_file,topics,out_folder)